## IFRS Taxonomy: Interactive ETL and Documentation

### Purpose of This Notebook

This Jupyter Notebook serves two primary roles:

1.  **Interactive Exploration**: It provides a step-by-step, visual walkthrough of the ETL (Extract, Transform, Load) process, allowing you to inspect the data at each stage.
2.  **Living Documentation**: It acts as the official documentation for the production-ready ETL script, [`ifrs_taxonomy_elt.py`](../src/etl/ifrs_taxonomy_elt.py), explaining the logic, business rules, and data modeling decisions.

> **Important**: This notebook is for demonstration and analysis only. All core business logic is implemented in the `.py` script and imported here. For automated or production runs, always execute the main script.

### The Goal

The objective is to transform the human-readable IFRS Taxonomy Excel file (`excel-taxonomy-iti-ifrs18-2025-by-fs.xlsx`) into structured, machine-readable formats. These outputs are designed to support relational databases, application backends, and analytics platforms.

---

### The Source Data 

The source file is designed for human readability and presents several challenges for direct machine processing:

1. **Section Headers**  
   Embedded headers (e.g., `[110000] General information...`) group related concepts. This grouping information must be extracted and propagated to all relevant child rows.

2. **Visual Hierarchy via Formatting**  
   Parent‑child relationships are indicated visually using Excel’s **cell indentation** (Format Cells → Alignment → Indent), not leading spaces in the text. This means the hierarchy depth must be read from the cell’s `alignment.indent` property, not by counting spaces.

3. **Implicit Data Types**  
   While most elements have an explicit `Type`, abstract (grouping) concepts are identified by a blank `Type` column. These must be explicitly labeled as `"abstract"`.

4. **Governance & Traceability**  
   For auditability, each row in the transformed data must retain its **original Excel row number** and group context (`group_code`, `group_name`).

---

### Data Modeling Approaches for Hierarchies

To represent the taxonomy's hierarchy in a flat table, the ETL process supports two complementary patterns:

| Approach | Adjacency List (`parent_concept`) | Materialized Path (`full_path` + `Level_X` columns) |
| :--- | :--- | :--- |
| **Structure** | Each row stores only its immediate parent's ID. | Each row stores its entire ancestry in a single `full_path` string and/or separate `Level_X` columns. |
| **Pros** | - Flexible (hierarchy can be infinitely deep).<br>- Normalized (no repeated data).<br>- Standard for relational backends (e.g., Django ORM). | - Very easy to query for analytics (e.g., `WHERE full_path LIKE '...%'`).<br>- Human‑readable; full context in one field.<br>- No recursion needed for descendant queries. |
| **Cons** | - Requires recursive queries to get all descendants. | - Fixed maximum depth if using `Level_X` columns.<br>- Data repetition in each row. |
| **Best For** | Application backends and transactional systems. | Analytics, reporting, search, and API payloads. |

---

### The Enhanced Solution

This ETL process: 

1. **Extracts**  
   - Reads the Excel file with `openpyxl` to capture both cell values and **indent levels**.  
   - Auto‑detects the `"Preferred label"` and `"Concept name"` columns by header name.  
   - Captures the **Excel row number** for governance.

2. **Transforms**  
   - **Section Headers** → Extracts `group_code` and `group_name` from header rows and forward‑fills them to children.  
   - **Clean Labels** → Strips whitespace, normalizes text.  
   - **Abstract Types** → Marks rows with blank `Type` and “[abstract]” in the label as `"abstract"`.  
   - **Adjacency List** → Builds `parent_concept` from indent levels.  
   - **Materialized Path** → Builds `Level_X` columns and a `full_path` string starting with `group_name` for uniqueness across sections.  
   - **Nested JSON Tree** → Generates a hierarchy structure for APIs and UI rendering.  
   - **Flattened Path Table** → Produces a “truly flat” table with one row per concept, including `full_path` and governance metadata.  
   - **Reverse Lookups** → Creates O(1) dictionaries to map `excel_row` or `concept_name` directly to full metadata.

3. **Loads**  
   - Outputs both the **nested JSON tree** and the **materialized path table** for downstream use.  
   - These outputs can be stored in a relational DB, served via an API, or used directly in analytics tools like Power BI.

---

### The Benefits

- **Migration‑safe**: Uses Excel’s actual formatting metadata, so hierarchy is preserved even if labels change.  
- **Governance‑ready**: Every node is traceable back to its exact Excel row and section.  
- **Dual‑model support**: Works for both backend systems (Adjacency List) and analytics/reporting (Materialized Path).  
- **API‑friendly**: JSON tree and reverse lookups make it easy to integrate into services without complex joins or recursion.

---
### Governance hooks
1. Every row in the DataFrame and every node in the JSON tree carries the original Excel row number for traceability.
2. JSON tree nodes carry a full path string from the root down to that node. This full path is globally unique by prepending the group_name (or group_code if you prefer) so that even if two sections have the same label structure, their paths won’t collide. This is especially useful for IFRS taxonomies where there are multiple statement sections (e.g., Statement of Financial Position, Statement of Profit or Loss) with similar sub‑trees.


## Notebook configuration

In [1]:
# --- Workbook configuration --
# System imports
import re
import json
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import logging
import sys
from pathlib import Path

# Set the project root directory (one level up from this notebook)
notebook_dir = Path.cwd()
project_root = notebook_dir.parent.resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Load black for code formatting in the notebook
%load_ext jupyter_black


# Import configurations
import config.etl_config as etl_config

# Setup a temporary logger for initial setup
temp_logger = etl_config.setup_logger(
        "etl_setup_notebook", project_root / "logs/etl_setup_notebook.log"
    )
temp_logger.info(f"Parent project root set to: {project_root}")

2025-11-06 19:31:32,827 - etl_setup_notebook - INFO - Logger 'etl_setup_notebook' initialized with level INFO


INFO:etl_setup_notebook:Logger 'etl_setup_notebook' initialized with level INFO


2025-11-06 19:31:32,828 - etl_setup_notebook - INFO - Parent project root set to: /Users/pri/Development/public/IFRS_taxonomy


INFO:etl_setup_notebook:Parent project root set to: /Users/pri/Development/public/IFRS_taxonomy


In [2]:
import src.etl.ifrs_taxonomy_elt as etl

In [3]:
# Load configuration
config = etl.load_config(project_root, temp_logger)
# Get paths
paths = etl.get_paths(config, project_root)
# Setup the main application logger using config paths
log_path = paths["logs_dir"] / "ifrs_taxonomy_etl.log"
logger = etl_config.setup_logger("ifrs_taxonomy_etl", log_path)

2025-11-06 19:31:32,878 - etl_setup_notebook - INFO - Config loaded from /Users/pri/Development/public/IFRS_taxonomy/config/etl_ifrs_taxonomy_pipeline_config.yaml


INFO:etl_setup_notebook:Config loaded from /Users/pri/Development/public/IFRS_taxonomy/config/etl_ifrs_taxonomy_pipeline_config.yaml


2025-11-06 19:31:32,879 - ifrs_taxonomy_etl - INFO - Logger 'ifrs_taxonomy_etl' initialized with level INFO


INFO:ifrs_taxonomy_etl:Logger 'ifrs_taxonomy_etl' initialized with level INFO


In [4]:
# log version info
logger.info(f"Python version: {sys.version}")
logger.info(f"pandas version: {pd.__version__}")
logger.info(f"openpyxl version: {openpyxl.__version__}")

2025-11-06 19:31:32,886 - ifrs_taxonomy_etl - INFO - Python version: 3.14.0 (v3.14.0:ebf955df7a8, Oct  7 2025, 08:20:14) [Clang 16.0.0 (clang-1600.0.26.6)]


INFO:ifrs_taxonomy_etl:Python version: 3.14.0 (v3.14.0:ebf955df7a8, Oct  7 2025, 08:20:14) [Clang 16.0.0 (clang-1600.0.26.6)]


2025-11-06 19:31:32,887 - ifrs_taxonomy_etl - INFO - pandas version: 2.3.3


INFO:ifrs_taxonomy_etl:pandas version: 2.3.3


2025-11-06 19:31:32,888 - ifrs_taxonomy_etl - INFO - openpyxl version: 3.1.5


INFO:ifrs_taxonomy_etl:openpyxl version: 3.1.5


## Extract Data with Indentation Handling
To reliably handle duplicate "Preferred Label" values, this ETL extracts both cell values and formatting (indent level) for each row using `openpyxl`, then builds the DataFrame row-by-row. This approach ensures that indentation and hierarchy are preserved for every row, even when labels repeat, by not relying on label uniqueness alone.

In [5]:
# --- Open IFRS Taxonomy Excel file ---

# File path and sheet name
file_path = (
    "/Users/pri/dev/learn/vsc_etl/ifrs/data/excel-taxonomy-iti-ifrs18-2025-by-fs.xlsx"
)
sheet_name = "Taxonomy ITI"

# Load workbook with openpyxl (to get indent levels)
wb = load_workbook(filename=file_path, data_only=True)
ws = wb[sheet_name]
logger.info(f"Loaded workbook: {file_path}, sheet: {sheet_name}")

# Read the first row to get column headers
headers = [
    cell.value for cell in next(ws.iter_rows(min_row=1, max_row=1, values_only=False))
]
logger.info(f"Extracted headers: {headers}")

# Finds the index (position) of the required columns:
preferred_label_idx = (
    headers.index("Preferred label") if "Preferred label" in headers else None
)
concept_name_idx = headers.index("Concept name") if "Concept name" in headers else None
type_idx = headers.index("Type") if "Type" in headers else None

if preferred_label_idx is None or concept_name_idx is None or type_idx is None:
    logger.error("One or more required columns are missing in the Excel sheet.")
    raise ValueError("Missing required columns in the Excel sheet.")
else:
    logger.info(
        f"Found column indices - Preferred Label: {preferred_label_idx}, Concept Name: {concept_name_idx}, Type: {type_idx}"
    )

2025-11-06 19:31:33,423 - ifrs_taxonomy_etl - INFO - Loaded workbook: /Users/pri/dev/learn/vsc_etl/ifrs/data/excel-taxonomy-iti-ifrs18-2025-by-fs.xlsx, sheet: Taxonomy ITI


INFO:ifrs_taxonomy_etl:Loaded workbook: /Users/pri/dev/learn/vsc_etl/ifrs/data/excel-taxonomy-iti-ifrs18-2025-by-fs.xlsx, sheet: Taxonomy ITI


2025-11-06 19:31:33,426 - ifrs_taxonomy_etl - INFO - Extracted headers: ['Concept name', 'Preferred label', 'Standard label', 'Documentation label', 'Guidance label', 'Type', 'References', 'Reference Links']


INFO:ifrs_taxonomy_etl:Extracted headers: ['Concept name', 'Preferred label', 'Standard label', 'Documentation label', 'Guidance label', 'Type', 'References', 'Reference Links']


2025-11-06 19:31:33,426 - ifrs_taxonomy_etl - INFO - Found column indices - Preferred Label: 1, Concept Name: 0, Type: 5


INFO:ifrs_taxonomy_etl:Found column indices - Preferred Label: 1, Concept Name: 0, Type: 5


In [6]:
# v2
# Optional metadata columns we want to capture
optional_cols = [
    "Standard label",
    "Documentation label",
    "Guidance label",
    "References",
    "Reference Links",
]

# Find indexes for all optional columns if present
col_indexes = {}
for col in optional_cols:
    col_indexes[col] = headers.index(col) if col in headers else None
logger.info(f"Optional column indices: {col_indexes}")

2025-11-06 19:31:33,436 - ifrs_taxonomy_etl - INFO - Optional column indices: {'Standard label': 2, 'Documentation label': 3, 'Guidance label': 4, 'References': 6, 'Reference Links': 7}


INFO:ifrs_taxonomy_etl:Optional column indices: {'Standard label': 2, 'Documentation label': 3, 'Guidance label': 4, 'References': 6, 'Reference Links': 7}


In [7]:
# Extract rows with indent + governance row number
rows = []
for excel_row_num, r in enumerate(ws.iter_rows(min_row=2, values_only=False), start=2):
    label_cell = r[preferred_label_idx]
    row_data = {
        "excel_row": excel_row_num,
        "Concept name": (
            r[concept_name_idx].value if concept_name_idx is not None else None
        ),
        "Preferred label": label_cell.value,
        "indent": int(label_cell.alignment.indent or 0),
        "Type": r[type_idx].value if type_idx is not None else None,
    }
    # v2: Add optional columns if present
    for col, idx in col_indexes.items():
        row_data[col] = r[idx].value if idx is not None else None
    rows.append(row_data)

In [8]:
# Convert rows to DataFrame
df_original = pd.DataFrame(rows)
logger.info(f"Converted rows to DataFrame. Shape: {df_original.shape}")

# Copy the original DataFrame to a shared review DataFrame for Data Wrangler, allowing inspection at any stage (using refresh) without reopening.
df_review = df_original.copy()

2025-11-06 19:31:33,514 - ifrs_taxonomy_etl - INFO - Converted rows to DataFrame. Shape: (7798, 10)


INFO:ifrs_taxonomy_etl:Converted rows to DataFrame. Shape: (7798, 10)


## Section 1: Process Group Headers

In [9]:
header_pattern = re.compile(r"^\[(\d{6})\]\s*(.*)")

df_grouped = df_original.copy()
df_grouped[["group_code", "group_name"]] = df_grouped["Concept name"].str.extract(
    header_pattern
)

# Forward-fill group info. Pandas ffill, fills missing values in a column with the last non-missing value above it
df_grouped["group_code"] = df_grouped["group_code"].ffill()
df_grouped["group_name"] = df_grouped["group_name"].ffill()

# Log the grouped DataFrame
logger.info(f"Grouped DataFrame created. Shape: {df_grouped.shape}")
# Copy the grouped DataFrame to the review DataFrame
df_review = df_grouped.copy()

2025-11-06 19:31:33,535 - ifrs_taxonomy_etl - INFO - Grouped DataFrame created. Shape: (7798, 12)


INFO:ifrs_taxonomy_etl:Grouped DataFrame created. Shape: (7798, 12)


## Section 2: Clean Labels and Set Abstract Type

In [10]:
df_cleaned = df_grouped.copy()

# Strip leading/trailing spaces from label
df_cleaned["label_clean"] = df_cleaned["Preferred label"].astype(str).str.strip()

# Identify abstract rows (Type is NaN or empty, AND label contains '[abstract]')
is_abstract = (
    df_cleaned["Type"].isna() | (df_cleaned["Type"].astype(str).str.strip() == "")
) & df_cleaned["label_clean"].str.contains("[abstract]", regex=False)
df_cleaned.loc[is_abstract, "Type"] = "abstract"

# Remove rows where Type is still NaN
df_cleaned = df_cleaned[df_cleaned["Type"].notna()].copy()

# Log the cleaned DataFrame
logger.info(f"Cleaned DataFrame created. Shape: {df_cleaned.shape}")

# Copy the cleaned DataFrame to the review DataFrame
df_review = df_cleaned.copy()

2025-11-06 19:31:33,560 - ifrs_taxonomy_etl - INFO - Cleaned DataFrame created. Shape: (7729, 13)


INFO:ifrs_taxonomy_etl:Cleaned DataFrame created. Shape: (7729, 13)


## Section 3: Build Hierarchical Structure

---

### Note on adjacency lists:
**Definition**: In an adjacency list, each node (concept) maintains a list of its direct children. This allows for efficient traversal and manipulation of the hierarchy.

#### Terminology and theory
1. **Root Node:**  
   The topmost parent in a hierarchy; every hierarchy starts with a root node.

2. **Leaf Node:**  
   The last node in a branch; it has no children.

3. **Internal/Intermediate Node:**  
   Any node that is neither a root nor a leaf; it has both a parent and at least one child.  
   *Both "internal node" and "intermediate node" are correct terms.*

4. **Hierarchy Aggregation:**  
   In analytics, you often aggregate all leaf nodes up to their parent, then those parents up to their own parent, and so on, until you reach the root.

#### Application in Current Setup

1. **parent = root node = 1:**  
   The top-level parent in each section is the root node.

2. **parent_concept missing:**  
   If `parent_concept` is blank (or `None`), that row is the highest in its section—i.e., the root node for that hierarchy. It is normal and correct for root nodes to have `parent_concept` as blank or `None`. This indicates they have no parent.

3. **parent_concept for each node:**  
   `parent_concept` gives the immediate parent for each node. The ultimate parent (root) will have `parent_concept` as blank/None.


**Summary Table:**

| Node Type         | parent_concept Value | Meaning                        |
|-------------------|---------------------|--------------------------------|
| Root node         | blank / None        | Top of hierarchy, no parent    |
| Internal node     | parent_concept set  | Has parent and children        |
| Leaf node         | parent_concept set  | Has parent, no children        |

### Note on Materialized Path

**Definition**: The Materialized Path is a way of representing hierarchical data by storing the full path to each node as a string.

#### Terminology and theory
1. Each row contains the full chain of its ancestors, representing its path in the hierarchy.
2. The hierarchy depth is indicated by the level numbers (Level_1, Level_2, ..., Level_X).

#### Application in Current Setup

1. The Materialized Path is constructed using the values in the `Concept name` column.
2. Columns `Level_1` to `Level_X` explicitly show each ancestor in the path. When a `Level_X` column is `None` or missing, it indicates that the row's concept is at that depth in the hierarchy.

In [11]:
def build_hierarchy(df, max_levels=5):
    parent_stack = []
    parent_concepts = []
    level_columns = {f"Level_{i + 1}": [] for i in range(max_levels)}

    for _, row in df.iterrows():
        level = row["indent"]
        concept_name = row["Concept name"]
        label = row["label_clean"]

        # Pop until correct parent
        while parent_stack and parent_stack[-1][0] >= level:
            parent_stack.pop()

        # Parent concept
        parent_concept = parent_stack[-1][1] if parent_stack else None
        parent_concepts.append(parent_concept)

        # Fill Level_X columns
        for i in range(max_levels):
            if i < len(parent_stack):
                level_columns[f"Level_{i + 1}"].append(parent_stack[i][2])
            else:
                level_columns[f"Level_{i + 1}"].append(None)

        # Push current node
        parent_stack.append((level, concept_name, label))

    df["parent_concept"] = parent_concepts
    for col, values in level_columns.items():
        df[col] = values
    return df


# Maximum value in the col indent
max_indent = df_cleaned["indent"].max()
logger.info(f"Maximum indent level found: {max_indent}")

df_hierarchy = build_hierarchy(df_cleaned, max_levels=max_indent)

# Log the hierarchical DataFrame
logger.info(f"Hierarchical DataFrame created. Shape: {df_hierarchy.shape}")

# Copy the hierarchical DataFrame to the review DataFrame
df_review = df_hierarchy.copy()

2025-11-06 19:31:33,585 - ifrs_taxonomy_etl - INFO - Maximum indent level found: 11


INFO:ifrs_taxonomy_etl:Maximum indent level found: 11


2025-11-06 19:31:33,698 - ifrs_taxonomy_etl - INFO - Hierarchical DataFrame created. Shape: (7729, 25)


INFO:ifrs_taxonomy_etl:Hierarchical DataFrame created. Shape: (7729, 25)


## Section 4: Convert to Nested JSON Tree with Governance + Full Path (group_name first)

In [12]:
def df_to_tree(df):
    root = []
    stack = [(-1, root, "")]  # (level, children_list, path_so_far)

    for _, row in df.iterrows():
        # Start path with group_name (or group_code if you prefer)
        base_path = row["group_name"] or row["group_code"] or ""
        current_path = base_path
        if stack[-1][2] and stack[-1][2] != base_path:
            current_path = stack[-1][2] + " > " + row["label_clean"]
        elif not stack[-1][2] or stack[-1][2] == base_path:
            current_path = base_path + (
                " > " + row["label_clean"] if row["label_clean"] else ""
            )

        # v2: Include all fields in the node
        node = {
            "excel_row": row["excel_row"],
            "concept_name": row["Concept name"],
            "preferred_label": row["Preferred label"],
            "label": row["label_clean"],
            "group_code": row["group_code"],
            "group_name": row["group_name"],
            "type": row["Type"],
            "full_path": current_path,
            "standard_label": row.get("Standard label"),
            "documentation_label": row.get("Documentation label"),
            "guidance_label": row.get("Guidance label"),
            "references": row.get("References"),
            "reference_links": row.get("Reference Links"),
            "children": [],
        }

        level = row["indent"]

        while stack and stack[-1][0] >= level:
            stack.pop()

        stack[-1][1].append(node)
        stack.append((level, node["children"], current_path))

    return root

In [13]:
taxonomy_tree = df_to_tree(df_hierarchy)
ifrs_taxonomy_tree = json.dumps(taxonomy_tree, indent=2, ensure_ascii=False)

In [14]:
# Log the nested JSON tree
logger.info(f"Nested JSON tree created with {len(taxonomy_tree)} top-level nodes.")
logger.info("JSON file created but not saved: name 'ifrs_taxonomy_tree.json'")
logger.info("JSON content preview (first 500 chars):")
logger.info(ifrs_taxonomy_tree[:500])

2025-11-06 19:31:39,705 - ifrs_taxonomy_etl - INFO - Nested JSON tree created with 225 top-level nodes.


INFO:ifrs_taxonomy_etl:Nested JSON tree created with 225 top-level nodes.


2025-11-06 19:31:39,706 - ifrs_taxonomy_etl - INFO - JSON file created but not saved: name 'ifrs_taxonomy_tree.json'


INFO:ifrs_taxonomy_etl:JSON file created but not saved: name 'ifrs_taxonomy_tree.json'


2025-11-06 19:31:39,706 - ifrs_taxonomy_etl - INFO - JSON content preview (first 500 chars):


INFO:ifrs_taxonomy_etl:JSON content preview (first 500 chars):


2025-11-06 19:31:39,707 - ifrs_taxonomy_etl - INFO - [
  {
    "excel_row": 3,
    "concept_name": "DisclosureOfGeneralInformationAboutFinancialStatementsExplanatory",
    "preferred_label": "Disclosure of general information about financial statements [text block]",
    "label": "Disclosure of general information about financial statements [text block]",
    "group_code": "110000",
    "group_name": "General information about financial statements",
    "type": "Text block",
    "full_path": "General information about financial statements > Disclo


INFO:ifrs_taxonomy_etl:[
  {
    "excel_row": 3,
    "concept_name": "DisclosureOfGeneralInformationAboutFinancialStatementsExplanatory",
    "preferred_label": "Disclosure of general information about financial statements [text block]",
    "label": "Disclosure of general information about financial statements [text block]",
    "group_code": "110000",
    "group_name": "General information about financial statements",
    "type": "Text block",
    "full_path": "General information about financial statements > Disclo


## Section 5: Flatten JSON to Materialized Path Table

In [15]:
def flatten_tree_to_table(tree):
    flat_rows = []

    def recurse(nodes):
        for node in nodes:
            # v2: Include all fields in the flat rows
            flat_rows.append(
                {
                    "full_path": node["full_path"],
                    "excel_row": node["excel_row"],
                    "concept_name": node["concept_name"],
                    "preferred_label": node["preferred_label"],
                    "label": node["label"],
                    "group_code": node["group_code"],
                    "group_name": node["group_name"],
                    "type": node["type"],
                    "standard_label": node.get("standard_label"),
                    "documentation_label": node.get("documentation_label"),
                    "guidance_label": node.get("guidance_label"),
                    "references": node.get("references"),
                    "reference_links": node.get("reference_links"),
                }
            )
            recurse(node["children"])

    recurse(tree)
    return pd.DataFrame(flat_rows)


df_materialized_path = flatten_tree_to_table(taxonomy_tree)

logger.info(f"Materialized Path DataFrame created. Shape: {df_materialized_path.shape}")

2025-11-06 19:31:39,744 - ifrs_taxonomy_etl - INFO - Materialized Path DataFrame created. Shape: (7729, 13)


INFO:ifrs_taxonomy_etl:Materialized Path DataFrame created. Shape: (7729, 13)


## Section 6: Build Reverse Lookup Dictionaries

In [16]:
lookup_by_excel_row = {
    row["excel_row"]: row for _, row in df_materialized_path.iterrows()
}

lookup_by_concept_name = {
    row["concept_name"]: row
    for _, row in df_materialized_path.iterrows()
    if pd.notna(row["concept_name"])
}

# Reverse Lookup Examples
logger.info("\n=== Reverse Lookup Examples ===")
print("----------------" * 2)
logger.info("By excel_row=17: %s", lookup_by_excel_row.get(17))
print("----------------" * 2)
logger.info(
    "By concept_name='PropertyPlantAndEquipment': %s",
    lookup_by_concept_name.get("PropertyPlantAndEquipment"),
)

2025-11-06 19:31:39,975 - ifrs_taxonomy_etl - INFO - 
=== Reverse Lookup Examples ===


INFO:ifrs_taxonomy_etl:
=== Reverse Lookup Examples ===


--------------------------------
2025-11-06 19:31:39,976 - ifrs_taxonomy_etl - INFO - By excel_row=17: full_path              General information about financial statements...
excel_row                                                             17
concept_name           PropertyPlantAndEquipmentIncludingRightofuseAs...
preferred_label        Property, plant and equipment including right-...
label                  Property, plant and equipment including right-...
group_code                                                        210000
group_name             Statement of financial position, current/non-c...
type                              Monetaryinstant, debit\n\t\t\t\t\t\t\t
standard_label         Property, plant and equipment including right-...
documentation_label    The amount of property, plant and equipment in...
guidance_label                                                          
references             IAS 16.73 e Common practice, IFRS 16.47 a Disc...
reference_links      

INFO:ifrs_taxonomy_etl:By excel_row=17: full_path              General information about financial statements...
excel_row                                                             17
concept_name           PropertyPlantAndEquipmentIncludingRightofuseAs...
preferred_label        Property, plant and equipment including right-...
label                  Property, plant and equipment including right-...
group_code                                                        210000
group_name             Statement of financial position, current/non-c...
type                              Monetaryinstant, debit\n\t\t\t\t\t\t\t
standard_label         Property, plant and equipment including right-...
documentation_label    The amount of property, plant and equipment in...
guidance_label                                                          
references             IAS 16.73 e Common practice, IFRS 16.47 a Disc...
reference_links        https://taxonomy.ifrs.org/xifrs-link?type=IAS&...
Name: 13, d

--------------------------------
2025-11-06 19:31:39,977 - ifrs_taxonomy_etl - INFO - By concept_name='PropertyPlantAndEquipment': full_path              General information about financial statements...
excel_row                                                           2558
concept_name                                   PropertyPlantAndEquipment
preferred_label           Property, plant and equipment at end of period
label                     Property, plant and equipment at end of period
group_code                                                        822100
group_name                         Notes - Property, plant and equipment
type                              Monetaryinstant, debit\n\t\t\t\t\t\t\t
standard_label                             Property, plant and equipment
documentation_label    The amount of tangible assets that: (a) are he...
guidance_label                                                          
references             IAS 16.73 e Disclosure, Effective 2027-01-0

INFO:ifrs_taxonomy_etl:By concept_name='PropertyPlantAndEquipment': full_path              General information about financial statements...
excel_row                                                           2558
concept_name                                   PropertyPlantAndEquipment
preferred_label           Property, plant and equipment at end of period
label                     Property, plant and equipment at end of period
group_code                                                        822100
group_name                         Notes - Property, plant and equipment
type                              Monetaryinstant, debit\n\t\t\t\t\t\t\t
standard_label                             Property, plant and equipment
documentation_label    The amount of tangible assets that: (a) are he...
guidance_label                                                          
references             IAS 16.73 e Disclosure, Effective 2027-01-01 I...
reference_links        https://taxonomy.ifrs.org/xifrs-l

## Outputs

In [17]:
# Output 1: DataFrame with hierarchy columns
logger.info("=== Output 1: DataFrame with hierarchy columns ===")
logger.info(f"Dataframe shape: {df_hierarchy.shape}")

2025-11-06 19:31:39,984 - ifrs_taxonomy_etl - INFO - === Output 1: DataFrame with hierarchy columns ===


INFO:ifrs_taxonomy_etl:=== Output 1: DataFrame with hierarchy columns ===


2025-11-06 19:31:39,985 - ifrs_taxonomy_etl - INFO - Dataframe shape: (7729, 25)


INFO:ifrs_taxonomy_etl:Dataframe shape: (7729, 25)


In [18]:
# Output 2: Nested JSON Tree
logger.info(f"Nested JSON tree created with {len(taxonomy_tree)} top-level nodes.")
logger.info("JSON content preview (first 500 chars):")
logger.info(json.dumps(taxonomy_tree, indent=2, ensure_ascii=False)[:500])

2025-11-06 19:31:39,993 - ifrs_taxonomy_etl - INFO - Nested JSON tree created with 225 top-level nodes.


INFO:ifrs_taxonomy_etl:Nested JSON tree created with 225 top-level nodes.


2025-11-06 19:31:39,994 - ifrs_taxonomy_etl - INFO - JSON content preview (first 500 chars):


INFO:ifrs_taxonomy_etl:JSON content preview (first 500 chars):


2025-11-06 19:31:45,878 - ifrs_taxonomy_etl - INFO - [
  {
    "excel_row": 3,
    "concept_name": "DisclosureOfGeneralInformationAboutFinancialStatementsExplanatory",
    "preferred_label": "Disclosure of general information about financial statements [text block]",
    "label": "Disclosure of general information about financial statements [text block]",
    "group_code": "110000",
    "group_name": "General information about financial statements",
    "type": "Text block",
    "full_path": "General information about financial statements > Disclo


INFO:ifrs_taxonomy_etl:[
  {
    "excel_row": 3,
    "concept_name": "DisclosureOfGeneralInformationAboutFinancialStatementsExplanatory",
    "preferred_label": "Disclosure of general information about financial statements [text block]",
    "label": "Disclosure of general information about financial statements [text block]",
    "group_code": "110000",
    "group_name": "General information about financial statements",
    "type": "Text block",
    "full_path": "General information about financial statements > Disclo


In [19]:
# Output 3: Materialized Path Table
logger.info("=== Output 3: Materialized Path Table ===")
logger.info(f"Dataframe shape: {df_materialized_path.shape}")

2025-11-06 19:31:45,886 - ifrs_taxonomy_etl - INFO - === Output 3: Materialized Path Table ===


INFO:ifrs_taxonomy_etl:=== Output 3: Materialized Path Table ===


2025-11-06 19:31:45,887 - ifrs_taxonomy_etl - INFO - Dataframe shape: (7729, 13)


INFO:ifrs_taxonomy_etl:Dataframe shape: (7729, 13)
